<a href="https://colab.research.google.com/github/shirinsitara/IMLO/blob/working-copy/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [723]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [724]:
#transform = transforms.ToTensor()
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),          # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

In [725]:
#train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [726]:
from torchvision.datasets import Flowers102

# Load the training data
train_data = Flowers102(root='/flower_data', split='train', download=True, transform=transform)

# Load the validation data
val_data = Flowers102(root='/flower_data', split='val', download=True, transform=transform)

# Load the test data
test_data = Flowers102(root='/flower_data', split='test', download=True, transform=transform)


In [727]:
train_data

Dataset Flowers102
    Number of datapoints: 1020
    Root location: /flower_data
    split=train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [728]:
test_data

Dataset Flowers102
    Number of datapoints: 6149
    Root location: /flower_data
    split=test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [729]:
val_data

Dataset Flowers102
    Number of datapoints: 1020
    Root location: /flower_data
    split=val
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [730]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [731]:
conv1 = nn.Conv2d(3, 10, 4, 2)
conv2 = nn.Conv2d(10, 16, 4, 1)


In [732]:
for i, (X_Train, y_train) in enumerate(train_data):
  break


In [733]:
X_Train.shape

torch.Size([3, 224, 224])

In [734]:
x = X_Train.view(1,3,224,224)

In [735]:
x = F.relu(conv1(x))

In [736]:
x.shape

torch.Size([1, 10, 111, 111])

In [737]:
x = F.max_pool2d(x,2,2)

In [738]:
x.shape

torch.Size([1, 10, 55, 55])

In [739]:
x = F.relu(conv2(x))

In [740]:
x.shape

torch.Size([1, 16, 52, 52])

In [741]:
x = F.avg_pool2d(x, 2, 2)

In [742]:
x.shape

torch.Size([1, 16, 26, 26])

In [743]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,10,4,2)
    self.conv2 = nn.Conv2d(10,16,4,1)
    # Fully Connected Layer
    self.fc1 = nn.Linear(26*26*16, 1200)
    self.fc2 = nn.Linear(1200, 840)
    self.fc3 = nn.Linear(840, 102)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2)
    # Second Pass
    X = F.relu(self.conv2(X))
    X = F.avg_pool2d(X,2,2)

    # Re-View to flatten it out
    X = X.view(-1, 26*26*16) # negative one so that we can vary the batch size

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)

In [744]:
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(3, 10, kernel_size=(4, 4), stride=(2, 2))
  (conv2): Conv2d(10, 16, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=10816, out_features=1200, bias=True)
  (fc2): Linear(in_features=1200, out_features=840, bias=True)
  (fc3): Linear(in_features=840, out_features=102, bias=True)
)

In [745]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Smaller the Learning Rate, longer its gonna take to train.

In [747]:
import time
start_time = time.time()

# Create Variables To Tracks Things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  # Train
  for b,(X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1
    y_pred = model(X_train) # get predicted values from the training set. Not flattened 2D
    loss = criterion(y_pred, y_train) # how off are we? Compare the predictions to correct answers in y_train

    predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the first point
    batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False=0, sum those up
    trn_corr += batch_corr # keep track as we go along in training.

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Print out some results
    if b%10 == 0:
      print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)


  # Test
  with torch.no_grad(): #No gradient so we don't update our weights and biases with test data
    for b,(X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] # Adding up correct predictions
      tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away


  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)



current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes!')

Epoch: 0  Batch: 10  Loss: 3.9854469299316406
Epoch: 0  Batch: 20  Loss: 4.384213447570801
Epoch: 0  Batch: 30  Loss: 4.4223127365112305
Epoch: 0  Batch: 40  Loss: 4.23541259765625
Epoch: 0  Batch: 50  Loss: 4.263166904449463
Epoch: 0  Batch: 60  Loss: 3.8584988117218018
Epoch: 0  Batch: 70  Loss: 3.8212294578552246
Epoch: 0  Batch: 80  Loss: 3.950770854949951
Epoch: 0  Batch: 90  Loss: 4.028338432312012
Epoch: 0  Batch: 100  Loss: 4.388976573944092
Epoch: 1  Batch: 10  Loss: 3.8314101696014404
Epoch: 1  Batch: 20  Loss: 3.946143388748169
Epoch: 1  Batch: 30  Loss: 4.07335901260376
Epoch: 1  Batch: 40  Loss: 3.680945634841919
Epoch: 1  Batch: 50  Loss: 4.26007604598999
Epoch: 1  Batch: 60  Loss: 4.0907368659973145
Epoch: 1  Batch: 70  Loss: 4.184187889099121
Epoch: 1  Batch: 80  Loss: 3.9348556995391846
Epoch: 1  Batch: 90  Loss: 4.031002521514893
Epoch: 1  Batch: 100  Loss: 3.6676411628723145
Epoch: 2  Batch: 10  Loss: 3.9018282890319824
Epoch: 2  Batch: 20  Loss: 3.7160308361053467
E

In [ ]:
train_losses = [tl.item() for tl in train_losses]
plt.plot(train_losses, label="Training Loss")
plt.plot(test_losses, label="Validation Loss")
plt.title("Loss at Epoch")
plt.legend()